In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [36]:
import sklearn

In [37]:
from sklearn.feature_extraction import DictVectorizer

In [10]:
import wget

In [11]:
wget.download('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

100% [........................................................................] 47673370 / 47673370

'yellow_tripdata_2023-01.parquet'

In [12]:
wget.download ('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

100% [........................................................................] 47748012 / 47748012

'yellow_tripdata_2023-02.parquet'

In [62]:
df = pd.read_parquet('./yellow_tripdata_2023-01.parquet')

In [53]:
df.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

In [63]:
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)

In [64]:
df['duration'] = df.tpep_dropoff_datetime-df.tpep_pickup_datetime

In [65]:
df.duration = df.duration.apply(lambda td:td.total_seconds()/60)

Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [57]:
std_dev_duration = df['duration'].std()
print(std_dev_duration)

42.59435124195458


Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [58]:
((df.duration >=1) & (df.duration <=60)).mean()

np.float64(0.9812202822125979)

Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

In [66]:
categorical = ['PULocationID','DOLocationID']
numerical = ['trip_distance']

In [60]:
df[categorical] = df[categorical].astype(str).dtypes

C:\Users\mayur\AppData\Local\Temp\ipykernel_1696\971723826.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df[categorical] = df[categorical].astype(str).dtypes


In [67]:
df[categorical + numerical]

,PULocationID,DOLocationID,trip_distance
0,161,141,0.97
1,43,237,1.10
2,48,238,2.51
3,138,7,1.90
4,107,79,1.43
...,...,...,...
3066761,107,48,3.05
3066762,112,75,5.80
3066763,114,239,4.67
3066764,230,79,3.15


In [68]:
train_dicts = df[categorical + numerical].to_dict(orient='records')

In [69]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [70]:
X_train

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9200298 stored elements and shape (3066766, 3)>

In [71]:
dv.feature_names_

['DOLocationID', 'PULocationID', 'trip_distance']

Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters, where duration is the response variable
Calculate the RMSE of the model on the training data
What's the RMSE on train?